A TensorFlow computation is described by a directed graph, which is composed of a set of nodes.

Clients typically construct a computational graph using one of the supported frontend languages (C++ or Python). 

![](img/graph.png)

In a TensorFlow graph, each node has zero or more inputs and zero or more outputs, and represents the instantiation of an operation. 

Values that flow along normal edges in the graph (from outputs to inputs) are <b>tensors</b>.

An <b>operation</b> has a name and represents an abstract computation (e.g., “matrix multiply”, or “add”).

Reference: Abadi, M., Agarwal, A., Barham, P., Brevdo, E., Chen, Z., Citro, C., Corrado, G.S., Davis, A., Dean, J., Devin, M. and Ghemawat, S., 2016. Tensorflow: Large-scale machine learning on heterogeneous distributed systems. arXiv preprint arXiv:1603.04467.

<strong> Logistic Regression Classification with TensorFlow </strong>

In [ ]:
import tensorflow as tf

The MNIST data is split into three parts: 55,000 data points of training data (mnist.train), 10,000 points of test data (mnist.test), and 5,000 points of validation data (mnist.validation). 
Each image is 28 pixels by 28 pixels. 

In [ ]:
# Import data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
trainX = mnist.train.images
trainY = mnist.train.labels
valX = mnist.validation.images
valY = mnist.validation.labels
testX = mnist.test.images
testY = mnist.test.labels
numFeatures = trainX.shape[1]
numLabels = trainY.shape[1]

<b>Implementing the model<b\>

<b>placeholders</b>:
x isn't a specific value. It's a placeholder, a value that we'll input when we ask TensorFlow to run a computation. We want to be able to input any number of MNIST images, each flattened into a 784-dimensional vector. We represent this as a 2-D tensor of floating-point numbers, with a shape [None, 784]. (Here None means that a dimension can be of any length.)

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, numFeatures])
y = tf.placeholder(tf.float32, [None, numLabels])     

A Variable is a modifiable tensor that lives in TensorFlow's graph of interacting operations. It can be used and even modified by the computation. 

In [ ]:
W = tf.Variable(tf.zeros([numFeatures, numLabels]))
b = tf.Variable(tf.zeros([numLabels]))

We can now construct our model. 

In [ ]:
logits = tf.nn.sigmoid(tf.matmul(x, W) + b)

<b> Training the model </b><br />
we ask TensorFlow to minimize cross_entropy cost using the gradient descent algorithm with a learning rate of 0.5. <br/>
TensorFlow also provides many other <a href="https://www.tensorflow.org/api_guides/python/train#Optimizers">optimization</a> algorithms

In [ ]:
# Training Parameters
learning_rate = 0.5
batch_size = 100
numEpochs=1000
avg_cost_val=[]
train_accuracy=[]
val_accuracy = []

In [ ]:
# Define cost & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

In [ ]:
# minimize cost
train_step = optimizer.minimize(cost)

Now, we create a tensorflow session:

In [ ]:
sess = tf.InteractiveSession()

We first have to create an operation to initialize the variables we created:

In [ ]:
tf.global_variables_initializer().run()

Evaluating Our Model

In [ ]:
# Evaluate model
prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

In [ ]:
# Start training
for epoch in range(numEpochs):
    print epoch
    avg_cost = 0.
    tr_avg_acc=0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for _ in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        _, c,tr_acc = sess.run([train_step,cost,accuracy], feed_dict={x: batch_xs, y: batch_ys})
        avg_cost += c / total_batch
        tr_avg_acc += tr_acc/total_batch
    # Display logs per epoch step
    avg_cost_val.append(avg_cost)
    train_accuracy.append(tr_avg_acc)
    val_accuracy.append(sess.run(accuracy, feed_dict={x:valX,y:valY}))
    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

<b> Plotting the cost, training and validation error <b/>

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(avg_cost_val,'r-',label = 'cost')
plt.figure()
plt.plot(train_accuracy,'k*-')            
plt.plot(val_accuracy,'bo-')
plt.legend(('training accuracy', 'validation accuracy'), shadow=True, fancybox=True)
plt.show()

Accuracy on <b> test data <b/>

In [ ]:
print(sess.run(accuracy, feed_dict={x:testX,y:testY}))

Don't forget to close the session

In [ ]:
sess.close()